# Práctica 1: Flujo de cargas (2025-GDEE)
Adaptación a python con PyPSA

In [1]:
import pandas as pd 
import numpy as np
import readFC
import re

In [2]:
from readFC import U_arg, extraer_potencia_linea, extraer_potencia_trafo
import numpy as np
import pandas as pd
import re

# Genera los nombres de archivo como strings: "5.3", "5.4", ..., "9.9"
archivos = [f"{x:.1f}" for x in np.arange(5.3, 9.9, 0.1)]

# Definir las líneas y el sentido de potencia a extraer
lineas_info = [
    {"numero": 1, "columna": 5, "sentido": "P(B->A)"},
    {"numero": 2, "columna": 5, "sentido": "P(B->A)"},
    {"numero": 3, "columna": 3, "sentido": "P(A->B)"}
]

# Lista para almacenar los datos de todos los archivos
todas_las_filas = []

for archivo in archivos:
    tensiones = []
    angulos = []
    pgen_nudo1 = None
    perdidas_activa = None
    potencias_lineas = []
    # Barrer todos los nudos (del 1 al 5) y almacenar resultados
    for n in range(1, 6):
        try:
            tension, angulo = U_arg(archivo, n)
        except Exception:
            tension, angulo = float('nan'), float('nan')
        tensiones.append(tension)
        angulos.append(angulo)
    
    # Extraer Pgen de Nudo1 y Pérdidas de activa
    try:
        with open(archivo, encoding="latin1") as f:
            lines = f.readlines()
        start = None
        for i, line in enumerate(lines):
            if "---------- RESULTADOS DEL FLUJO DE CARGAS ----------" in line:
                start = i
                break
        if start is not None:
            for line in lines[start:]:
                if line.strip().startswith("Nudo1"):
                    partes = line.split()
                    if len(partes) > 4:
                        pgen_nudo1 = float(partes[4])
                # Buscar pérdidas de activa
                match = re.search(r"de activa=\s*([\d\.\-Ee]+)", line)
                if match:
                    perdidas_activa = float(match.group(1))
    except Exception:
        pgen_nudo1 = float('nan')
        perdidas_activa = float('nan')
    
    # Extraer potencias de las líneas indicadas
    for info in lineas_info:
        try:
            valor = extraer_potencia_linea(archivo, info["numero"], info["columna"])
        except Exception:
            valor = float('nan')
        potencias_lineas.append(valor)
    
    # Extraer P(Prim.) de Trafo1 y Trafo2
    try:
        p_prim_1 = extraer_potencia_trafo(archivo, 1, 3)
    except Exception:
        p_prim_1 = float('nan')
    try:
        p_prim_2 = extraer_potencia_trafo(archivo, 2, 3)
    except Exception:
        p_prim_2 = float('nan')
    
    # Construir la fila para este archivo
    fila = []
    for n in range(1, 6):
        fila.append(tensiones[n-1])
        fila.append(angulos[n-1])
    fila.append(pgen_nudo1)
    fila.append(perdidas_activa)
    fila.extend(potencias_lineas)
    fila.append(p_prim_1)
    fila.append(p_prim_2)
    todas_las_filas.append(fila)

# Crear columnas: U1, θ1, ..., U5, θ5, Pgen_Nudo1, Perdidas_Activa, P(B->A)_L1, P(B->A)_L2, P(A->B)_L3, P(Prim.)_Trafo1, P(Prim.)_Trafo2
columnas = []
for n in range(1, 6):
    columnas.append(f"U{n}")
    columnas.append(f"θ{n}")
columnas.append("Pgen_Nudo1")
columnas.append("Perdidas_Activa")
for info in lineas_info:
    columnas.append(f"{info['sentido']}_Linea{info['numero']}")
columnas.append("P(Prim.)_Trafo1")
columnas.append("P(Prim.)_Trafo2")

# Crear el DataFrame final
df = pd.DataFrame(todas_las_filas, columns=columnas, index=archivos)
df.index.name = "Archivo"

# Guardar en Excel
# df.to_excel("tensiones_angulos_varios.xlsx")
# print(df) # visualizar por pantalla